In [1]:
import numpy as np
import pandas as pd

In [2]:
df_gods=pd.read_csv("/content/greek_gods.csv")
df_godesses=pd.read_csv("/content/greek_goddesses.csv")

In [3]:
df_gods['Gender'] = 'God'
df_godesses['Gender']='Godess'

In [4]:
df_gods

,Name,Age,Domain,Gender
0,Zeus,6000,Olympian,God
1,Poseidon,5500,Sea,God
2,Hades,5000,Underworld,God
3,Ares,4000,War,God
4,Hephaestus,3500,Fire & Metalworking,God
5,Chronos,10000,Time,God
6,Apollo,8200,"Music, Poetry, Light, Healing",God


In [5]:
df_godesses

,Name,Age,Domain,Gender
0,Hera,6500,Olympian,Godess
1,Aphrodite,4500,Love & Beauty,Godess
2,Athena,7000,Wisdom & Crafts,Godess
3,Demeter,8500,Agriculture,Godess
4,Hestia,3000,Hearth & Home,Godess
5,Eris,8000,War,Godess


1. Merge the data from greek_gods.csv and greek_goddesses.csv based on a common field and create a new table that includes information about both gods and goddesses.


In [6]:
df = pd.concat([df_gods, df_godesses], ignore_index=True)

In [7]:
df

,Name,Age,Domain,Gender
0,Zeus,6000,Olympian,God
1,Poseidon,5500,Sea,God
2,Hades,5000,Underworld,God
3,Ares,4000,War,God
4,Hephaestus,3500,Fire & Metalworking,God
5,Chronos,10000,Time,God
6,Apollo,8200,"Music, Poetry, Light, Healing",God
7,Hera,6500,Olympian,Godess
8,Aphrodite,4500,Love & Beauty,Godess
9,Athena,7000,Wisdom & Crafts,Godess


2. Filter the merged table to only include gods and goddesses who are older than 8000 years, then sort them based on their ages in descending order.


In [8]:
df2 =df[df['Age'] > 8000].sort_values(by='Age', ascending=False)

print(df2)

       Name    Age                         Domain  Gender
5   Chronos  10000                           Time     God
10  Demeter   8500                    Agriculture  Godess
6    Apollo   8200  Music, Poetry, Light, Healing     God


3. Join the two tables based on the "Domain" field and calculate the average age of gods and goddesses in each domain.

In [21]:
# Merge the two tables based on the "Domain" field
df3 = pd.merge(df_gods, df_godesses, on='Domain', suffixes=('_god', '_goddess'), how='outer')

# Combine "Age" and "Gender" columns into single columns
df3['Age'] = df3['Age_god'].fillna(df3['Age_goddess'])
df3['Gender'] = 'God'  # Default value for gender
df3.loc[df3['Age_goddess'].notna(), 'Gender'] = 'Goddess'  # Update gender for goddesses
df3['Name'] = df3['Name_god'].fillna(df3['Name_goddess'])  # Combine name columns
df3.drop(['Name_god', 'Name_goddess', 'Age_god', 'Age_goddess', 'Gender_god', 'Gender_goddess'], axis=1, inplace=True)  # Drop redundant columns

# Reposition the "Name" column to be the first column
cols = list(df3.columns)
cols.remove('Name')
df3 = df3[['Name'] + cols]
df3



,Name,Domain,Age,Gender
0,Zeus,Olympian,6000.0,Goddess
1,Poseidon,Sea,5500.0,God
2,Hades,Underworld,5000.0,God
3,Ares,War,4000.0,Goddess
4,Hephaestus,Fire & Metalworking,3500.0,God
5,Chronos,Time,10000.0,God
6,Apollo,"Music, Poetry, Light, Healing",8200.0,God
7,Aphrodite,Love & Beauty,4500.0,Goddess
8,Athena,Wisdom & Crafts,7000.0,Goddess
9,Demeter,Agriculture,8500.0,Goddess


In [22]:
# Calculate the average age of gods and goddesses in each domain
avg_age_per_domain = df3.groupby('Domain')['Age'].mean()

print(avg_age_per_domain)

Domain
Agriculture                       8500.0
Fire & Metalworking               3500.0
Hearth & Home                     3000.0
Love & Beauty                     4500.0
Music, Poetry, Light, Healing     8200.0
Olympian                          6000.0
Sea                               5500.0
Time                             10000.0
Underworld                        5000.0
War                               4000.0
Wisdom & Crafts                   7000.0
Name: Age, dtype: float64


4. Determine which god/goddess has the highest age, and then find out if they are a god or a goddess.


In [23]:
# Find the entity with the highest age
max_age_entity = df3.loc[df3['Age'].idxmax()]

# Determine if the entity is a god or a goddess
if max_age_entity['Gender'] == 'God':
    print(f"The entity with the highest age is {max_age_entity['Name']} who is a god.")
elif max_age_entity['Gender'] == 'Goddess':
    print(f"The entity with the highest age is {max_age_entity['Name']} who is a goddess.")
else:
    print("No entity found with the highest age.")

The entity with the highest age is Chronos who is a god.


5. Create a new column in each table called "Age Group" and categorize the gods/goddesses into groups such as "Young" (age < 5000), "Middle-aged" (age between 5000 and 8000), and "Old" (age > 8000)


In [24]:
# Define a function to categorize age into groups
def categorize_age(age):
    if age < 5000:
        return 'Young'
    elif 5000 <= age < 8000:
        return 'Middle-aged'
    elif age >= 8000:
        return 'Old'
    else:
        return 'Unknown'

# Apply the function to create the "Age Group" column
df_gods['Age Group'] = df_gods['Age'].apply(categorize_age)
df_godesses['Age Group'] = df_godesses['Age'].apply(categorize_age)

# Display the updated tables
print("Table for Gods:")
print(df_gods)
print("\nTable for Goddesses:")
print(df_godesses)


Table for Gods:
         Name    Age                         Domain Gender    Age Group
0        Zeus   6000                       Olympian    God  Middle-aged
1    Poseidon   5500                            Sea    God  Middle-aged
2       Hades   5000                     Underworld    God  Middle-aged
3        Ares   4000                            War    God        Young
4  Hephaestus   3500            Fire & Metalworking    God        Young
5     Chronos  10000                           Time    God          Old
6      Apollo   8200  Music, Poetry, Light, Healing    God          Old

Table for Goddesses:
        Name   Age           Domain  Gender    Age Group
0       Hera  6500         Olympian  Godess  Middle-aged
1  Aphrodite  4500    Love & Beauty  Godess        Young
2     Athena  7000  Wisdom & Crafts  Godess  Middle-aged
3    Demeter  8500      Agriculture  Godess          Old
4     Hestia  3000    Hearth & Home  Godess        Young
5       Eris  8000              War  Godess 

6. Compare the average ages of gods and goddesses. Is there a significant age difference between them? If yes, which group tends to be older?


In [25]:
# Calculate the average age of gods and goddesses separately
avg_age_gods = df_gods['Age'].mean()
avg_age_goddesses = df_godesses['Age'].mean()

# Determine if there is a significant age difference
if avg_age_gods > avg_age_goddesses:
    print("Gods tend to be older than goddesses.")
    print(f"The average age of gods is {avg_age_gods:.2f} years.")
    print(f"The average age of goddesses is {avg_age_goddesses:.2f} years.")
elif avg_age_gods < avg_age_goddesses:
    print("Goddesses tend to be older than gods.")
    print(f"The average age of gods is {avg_age_gods:.2f} years.")
    print(f"The average age of goddesses is {avg_age_goddesses:.2f} years.")
else:
    print("There is no significant age difference between gods and goddesses.")


Goddesses tend to be older than gods.
The average age of gods is 6028.57 years.
The average age of goddesses is 6250.00 years.


7. Write a Python program using for loop to iterate over the "Age" column of the merged table (after merging the gods and goddesses tables) and print out the names of gods/goddesses who are older than 8000 years


In [26]:
# Iterate over the "Age" column of the merged table
for index, row in df3.iterrows():
    if row['Age'] > 8000:
        print(f"{row['Name']} is older than 8000 years.")


Chronos is older than 8000 years.
Apollo is older than 8000 years.
Demeter is older than 8000 years.


8. Write a Python program to find the oldest god/goddess from the merged table Lafter merging the gods and goddesses tables) by iterating through the "Age" column using a while loop Print out the name of the oldest god/goddess and their age

In [27]:
# Initialize variables to store the oldest age and corresponding name
oldest_age = 0
oldest_name = ""

# Iterate through the "Age" column using a while loop
index = 0
while index < len(df3):
    current_age = df3.iloc[index]['Age']
    current_name = df3.iloc[index]['Name']
    if current_age > oldest_age:
        oldest_age = current_age
        oldest_name = current_name
    index += 1

# Print out the name of the oldest god/goddess and their age
print(f"The oldest god/goddess is {oldest_name} with an age of {oldest_age} years.")


The oldest god/goddess is Chronos with an age of 10000.0 years.
